In [3]:
import json, os, requests

from key import API_KEY

API_KEY = API_KEY # Enter your API Key/"App ID" Here. Mine was 40 chars long.

# FILM_CAMS = '15230'
# ELEC_GUITARS = '33034' # Category code for electric guitars on eBay.
# WATCHES = '14324'
MENS_SUNGLASSES = '79720'

In [4]:
USA = 'EBAY-US' # USA Marketplace only. API seems to start returning stuff from other markets eventually anyhow
USED = '3000' # Just the code for item condition "used". This is the focus of this project.
AUCTIONWITHBIN = "AuctionWithBIN"
AUCTION = "Auction"

ITEM_FILTER_0 = f'itemFilter(0).name=Condition&itemFilter(0).value={USED}' # Only used guitars
ITEM_FILTER_1 = f'itemFilter(1).name=HideDuplicateItems&itemFilter(1).value=true' # No duplicate listings
ITEM_FILTER_2 = f'itemFilter(2).name=MinPrice&itemFilter(2).value=49' # Only items that sold for > this value
ITEM_FILTER_3 = f'itemFilter(3).name=MaxQuantity&itemFilter(3).value=1' # No lots or batch sales. One item at a time
ITEM_FILTER_4 = f'itemFilter(4).name=SoldItemsOnly&itemFilter(4).value=true' # Only looking at listings that sold.
ITEM_FILTER_5 = f'itemFilter(5).name=ListingType&itemFilter(5).value={AUCTION}' # Select sale type.
ITEM_FILTER_6 = f'itemFilter(6).name=MaxPrice&itemFilter(6).value=2000' # Only items that sold for < this value
ITEM_FILTER_7 = f'itemFilter(7).name=listingType&itemFilter(7).value={AUCTIONWITHBIN}' # Select sale type.

FIND_COMPLETED = 'findCompletedItems' # This is the eBay API endpoint service we'll be querying.

In [5]:
def find_completed_auction(PAGE):
    '''Make a request to the eBay API and return the JSON text of this page number'''
    r = requests.get(f'https://svcs.ebay.com/services/search/FindingService/v1?'
                 f'OPERATION-NAME={FIND_COMPLETED}&'
                 f'X-EBAY-SOA-SECURITY-APPNAME={API_KEY}&'
                 f'RESPONSE-DATA-FORMAT=JSON&' # This value can be altered if you're not into JSON responses
                 f'REST-PAYLOAD&'
                 f'GLOBAL-ID={USA}&' # seems to prioritize the value you enter but returns other stuff too
                 f'categoryId={MENS_SUNGLASSES}&' # Product category goes here
                 f'descriptionSearch=true&' # More verbose responses
                 f'{ITEM_FILTER_0}&' # USED
                 f'{ITEM_FILTER_1}&' # NO DUPES
                 f'{ITEM_FILTER_2}&' # MINPRICE
                 f'{ITEM_FILTER_3}&' # NO LOTS
                 f'{ITEM_FILTER_4}&' # ONLY SOLD
                 f'{ITEM_FILTER_5}&' # AUCTIONS ONLY
                 f'{ITEM_FILTER_6}&' # MAX PRICE
                 f'paginationInput.pageNumber={str(PAGE)}&' # value to be looped through when collecting lotsa data
                 f'outputSelector=PictureURLLarge') # Why not grab the thumbnail URLs too. Could be cool
    if r.json()['findCompletedItemsResponse'][0].get('searchResult'):
        return r.json()['findCompletedItemsResponse'][0]['searchResult'][0]['item']
    else:
        return None

In [6]:
# For BUY IT NOW:
def find_completed_auction_BIN(PAGE):
    '''Make a request to the eBay API and return the JSON text of this page number'''
    r = requests.get(f'https://svcs.ebay.com/services/search/FindingService/v1?'
                 f'OPERATION-NAME={FIND_COMPLETED}&'
                 f'X-EBAY-SOA-SECURITY-APPNAME={API_KEY}&'
                 f'RESPONSE-DATA-FORMAT=JSON&' # This value can be altered if you're not into JSON responses
                 f'REST-PAYLOAD&'
                 f'GLOBAL-ID={USA}&' # seems to prioritize the value you enter but returns other stuff too
                 f'categoryId={MENS_SUNGLASSES}&' # Product category goes here
                 f'descriptionSearch=true&' # More verbose responses
                 f'{ITEM_FILTER_0}&' # USED
                 f'{ITEM_FILTER_1}&' # NO DUPES
                 f'{ITEM_FILTER_2}&' # MINPRICE
                 f'{ITEM_FILTER_3}&' # NO LOTS
                 f'{ITEM_FILTER_4}&' # ONLY SOLD
                 f'{ITEM_FILTER_7}&' # BUY IT NOW ONLY
                 f'paginationInput.pageNumber={str(PAGE)}&' # value to be looped through when collecting lotsa data
                 f'outputSelector=PictureURLLarge') # Why not grab the thumbnail URLs too. Could be cool
    if r.json()['findCompletedItemsResponse'][0].get('searchResult'):
        return r.json()['findCompletedItemsResponse'][0]['searchResult'][0]['item']
    else:
        return None

In [13]:
def get_specs(ITEM_ID):
    '''Return the specifics of a single eBay auction. String input.'''
    r2 = requests.get('http://open.api.ebay.com/shopping?'
                    f'callname=GetSingleItem&'
                    f'responseencoding=JSON&'
                    f'appid={API_KEY}&'
                    f'siteid=0&' # USA Store
                    f'version=967&' # What is this?
                    f'ItemID={ITEM_ID}&' # Assigned above
                    f'IncludeSelector=Details,ItemSpecifics,TextDescription')
    try:
        return r2.json()['Item']
    except KeyError:
        pass

In [14]:
# Seems like there's some variability between items when it comes to item specifics field.
# ### Persisting some Data for Analysis

# Just write first page of listing results to .json files:
def persist_page_to_json(PAGE):
    '''Saves a page of JSON responses to one json per sale / item'''
    for i in range(len(PAGE)):
        with open("data/listings/%s_listing.json" % (PAGE[i]['itemId'][0]), 'w') as f:  # writing JSON object
            json.dump(PAGE[i], f)

In [15]:
# Now write one page of details to a JSON:
def persist_spec_to_json(spec):
    '''Writes one page of Axe Specs to one json'''
    try:
        with open("data/specs/%s_specs.json" % (spec['ItemID']), 'w') as f:  # writing JSON object
            json.dump(spec, f)
    except TypeError:
        pass
    pass

In [16]:
# Okay, careful, this is where we start to hammer the eBay API a little bit.
def spam_the_api(start_page, stop_page, fetch_function):
    existing_files = [name.split('_')[0] for name in os.listdir('data/specs/') if not name.startswith('.')] # Ignore .DS_Store
    
    j = 0
    k = 0
    
    '''Spams the eBay API for pages of item DATA'''
    
    for i in range(start_page+1, stop_page+1):
        page = fetch_function(i)
        if page != None:
            persist_page_to_json(page)
            for item in page:
                k += 1
                if item['itemId'][0] not in existing_files:
                    j += 1
                    print(f'Get page {i} item {k}')
                    persist_spec_to_json(get_specs(item['itemId'][0]))
                else:
                    print(f'Skip page {i} item {k}')    
    print(f'\nChecked {k} items')
    print(f'\nGot {j} new items')

In [ ]:
# Again, you only get 5k API calls per day.
spam_the_api(0,20, find_completed_auction)

Skip page 1 item 1
Skip page 1 item 2
Skip page 1 item 3
Skip page 1 item 4
Skip page 1 item 5
Skip page 1 item 6
Skip page 1 item 7
Skip page 1 item 8
Skip page 1 item 9
Skip page 1 item 10
Skip page 1 item 11
Skip page 1 item 12
Skip page 1 item 13
Skip page 1 item 14
Skip page 1 item 15
Skip page 1 item 16
Skip page 1 item 17
Skip page 1 item 18
Skip page 1 item 19
Skip page 1 item 20
Skip page 1 item 21
Skip page 1 item 22
Skip page 1 item 23
Skip page 1 item 24
Skip page 1 item 25
Skip page 1 item 26
Skip page 1 item 27
Skip page 1 item 28
Skip page 1 item 29
Skip page 1 item 30
Get page 1 item 31
Get page 1 item 32
Skip page 1 item 33
Skip page 1 item 34
Skip page 1 item 35
Skip page 1 item 36
Skip page 1 item 37
Skip page 1 item 38
Skip page 1 item 39
Skip page 1 item 40
Skip page 1 item 41
Skip page 1 item 42
Skip page 1 item 43
Skip page 1 item 44
Skip page 1 item 45
Skip page 1 item 46
Skip page 1 item 47
Skip page 1 item 48
Skip page 1 item 49
Skip page 1 item 50
Skip page 1

Skip page 5 item 401
Skip page 5 item 402
Skip page 5 item 403
Skip page 5 item 404
Skip page 5 item 405
Skip page 5 item 406
Skip page 5 item 407
Skip page 5 item 408
Skip page 5 item 409
Skip page 5 item 410
Skip page 5 item 411
Skip page 5 item 412
Skip page 5 item 413
Skip page 5 item 414
Skip page 5 item 415
Skip page 5 item 416
Skip page 5 item 417
Skip page 5 item 418
Skip page 5 item 419
Skip page 5 item 420
Skip page 5 item 421
Skip page 5 item 422
Skip page 5 item 423
Skip page 5 item 424
Skip page 5 item 425
Skip page 5 item 426
Skip page 5 item 427
Skip page 5 item 428
Skip page 5 item 429
Skip page 5 item 430
Skip page 5 item 431
Skip page 5 item 432
Skip page 5 item 433
Skip page 5 item 434
Skip page 5 item 435
Skip page 5 item 436
Skip page 5 item 437
Skip page 5 item 438
Skip page 5 item 439
Skip page 5 item 440
Skip page 5 item 441
Skip page 5 item 442
Skip page 5 item 443
Skip page 5 item 444
Skip page 5 item 445
Skip page 5 item 446
Skip page 5 item 447
Skip page 5 i

Skip page 9 item 801
Skip page 9 item 802
Skip page 9 item 803
Skip page 9 item 804
Skip page 9 item 805
Skip page 9 item 806
Skip page 9 item 807
Skip page 9 item 808
Skip page 9 item 809
Skip page 9 item 810
Skip page 9 item 811
Skip page 9 item 812
Skip page 9 item 813
Skip page 9 item 814
Skip page 9 item 815
Skip page 9 item 816
Skip page 9 item 817
Skip page 9 item 818
Skip page 9 item 819
Skip page 9 item 820
Skip page 9 item 821
Skip page 9 item 822
Skip page 9 item 823
Skip page 9 item 824
Skip page 9 item 825
Skip page 9 item 826
Skip page 9 item 827
Skip page 9 item 828
Skip page 9 item 829
Skip page 9 item 830
Skip page 9 item 831
Skip page 9 item 832
Skip page 9 item 833
Skip page 9 item 834
Skip page 9 item 835
Skip page 9 item 836
Skip page 9 item 837
Skip page 9 item 838
Skip page 9 item 839
Skip page 9 item 840
Skip page 9 item 841
Skip page 9 item 842
Skip page 9 item 843
Skip page 9 item 844
Skip page 9 item 845
Skip page 9 item 846
Skip page 9 item 847
Skip page 9 i

Get page 12 item 1181
Get page 12 item 1182
Get page 12 item 1183
Get page 12 item 1184
Get page 12 item 1185
Get page 12 item 1186
Get page 12 item 1187
Get page 12 item 1188


In [ ]:
spam_the_api(0, 20, find_completed_auction_BIN)